## Building a LAS file from scratch


In [ ]:
import sphinx

In [1]:
import numpy as np
import lasio
import numpy as np

In [2]:
lasbuilt = lasio.LASFile()

In [3]:
lasbuilt.header

{'Version': [HeaderItem(mnemonic=VERS, unit=, value=2.0, descr=CWLS log ASCII Standard -V...),
  HeaderItem(mnemonic=WRAP, unit=, value=NO, descr=One line per depth step),
  HeaderItem(mnemonic=DLM, unit=, value=SPACE, descr=Column Data Section Delim...)],
 'Well': [HeaderItem(mnemonic=STRT, unit=m, value=nan, descr=START DEPTH),
  HeaderItem(mnemonic=STOP, unit=m, value=nan, descr=STOP DEPTH),
  HeaderItem(mnemonic=STEP, unit=m, value=nan, descr=STEP),
  HeaderItem(mnemonic=NULL, unit=, value=-9999.25, descr=NULL VALUE),
  HeaderItem(mnemonic=COMP, unit=, value=, descr=COMPANY),
  HeaderItem(mnemonic=WELL, unit=, value=, descr=WELL),
  HeaderItem(mnemonic=FLD, unit=, value=, descr=FIELD),
  HeaderItem(mnemonic=LOC, unit=, value=, descr=LOCATION),
  HeaderItem(mnemonic=PROV, unit=, value=, descr=PROVINCE),
  HeaderItem(mnemonic=CNTY, unit=, value=, descr=COUNTY),
  HeaderItem(mnemonic=STAT, unit=, value=, descr=STATE),
  HeaderItem(mnemonic=CTRY, unit=, value=, descr=COUNTRY),
  Header

In [4]:
from datetime import datetime

In [5]:
lasbuilt.well.DATE = str(datetime.today())

In [6]:
lasbuilt.other = 'Example of how to create a LAS file from scratch using lasio'

In [7]:
depths = np.arange(10, 50, 0.5)

In [8]:
synth = np.log10(depths)*5+np.random.random(len(depths))

In [9]:
synth[:8] = np.nan

In [10]:
lasbuilt.add_curve('DEPT', depths, unit='m')
lasbuilt.add_curve('SYNTH', synth, descr='fake data')

In [11]:
las12 = lasbuilt.write('scratch_v1.2.las', version=1.2)

lasbuilt.write('scratch_v2.las', version=2)

## Integration with pandas.DataFrame


### Structure of a as file

1. VERSION INFORMATION
2. WELL INFORMATION
  * 
3. CURVE INFORMATION
  * DEPT[M]
  * CALI[MM]
  * DFAR.G/CM3
  * DNEAR.G/CM3
  * GAMN.GAPI                
  * NEUT.CPS                 
  * PR.OHM/M
  * SP.MV
  * COND.MS/M                
4. PARAMETER INFORMATION
5. OTHER

In [12]:
las = lasio.read("6038187_v1.2.las")

In [13]:
df = las.df()

In [14]:
df.head

<bound method NDFrame.head of           CALI   DFAR  DNEAR     GAMN  NEUT       PR     SP     COND
DEPT                                                                
0.05    49.765  4.587  3.382      NaN   NaN      NaN    NaN      NaN
0.10    49.765  4.587  3.382 -2324.28   NaN  115.508 -3.049 -116.998
0.15    49.765  4.587  3.382 -2324.28   NaN  115.508 -3.049 -116.998
0.20    49.765  4.587  3.382 -2324.28   NaN  115.508 -3.049 -116.998
0.25    49.765  4.587  3.382 -2324.28   NaN  115.508 -3.049 -116.998
...        ...    ...    ...      ...   ...      ...    ...      ...
136.40  48.604    NaN    NaN      NaN   NaN      NaN    NaN      NaN
136.45  48.555    NaN    NaN      NaN   NaN      NaN    NaN      NaN
136.50  48.555    NaN    NaN      NaN   NaN      NaN    NaN      NaN
136.55  48.438    NaN    NaN      NaN   NaN      NaN    NaN      NaN
136.60 -56.275    NaN    NaN      NaN   NaN      NaN    NaN      NaN

[2732 rows x 8 columns]>

In [15]:
df.describe()

,CALI,DFAR,DNEAR,GAMN,NEUT,PR,SP,COND
count,2732.000000,2701.000000,2701.000000,2691.000000,2492.000000,2692.000000,2692.000000,2697.000000
mean,97.432002,1.767922,1.729209,-102.330033,441.600013,17940.522307,90.393464,478.670791
std,13.939547,0.480333,0.372412,630.106420,370.138208,22089.297212,26.725547,753.869866
min,-56.275000,0.725000,0.657001,-2324.280000,81.001800,115.508000,-3.049000,-116.998000
25%,101.077500,1.526000,1.535000,55.783000,158.002000,2652.470000,93.495500,200.981000
50%,101.426000,1.758000,1.785000,74.376900,256.501500,2709.345000,99.994000,266.435000
75%,101.582000,1.993000,1.948000,88.326900,680.500250,50499.900000,100.623000,505.530000
max,103.380000,5.989000,3.382000,169.672000,1665.990000,50499.900000,102.902000,4978.160000


*There’s obviously a problem with the GAMN log: -2324.28 is not a valid value. Let’s fix that.*



In [16]:
df['GAMN'][df['GAMN'] == -2324.28] = np.nan

In [17]:
df.describe()['GAMN']

count    2491.000000
mean       76.068198
std        23.120160
min        13.946000
25%        60.434100
50%        76.700700
75%        90.647500
max       169.672000
Name: GAMN, dtype: float64

*Let us create a new field*

In [18]:
df['GAMN_avg'] = df['GAMN'].rolling(int(1 / las.well.STEP.value), center=True).mean()

In [19]:
las.curves

[CurveItem(mnemonic=DEPT, unit=M, value=, descr=DEPTH, original_mnemonic=DEPT, data.shape=(2732,)),
 CurveItem(mnemonic=CALI, unit=MM, value=, descr=CALI, original_mnemonic=CALI, data.shape=(2732,)),
 CurveItem(mnemonic=DFAR, unit=G/CM3, value=, descr=DFAR, original_mnemonic=DFAR, data.shape=(2732,)),
 CurveItem(mnemonic=DNEAR, unit=G/CM3, value=, descr=DNEAR, original_mnemonic=DNEAR, data.shape=(2732,)),
 CurveItem(mnemonic=GAMN, unit=GAPI, value=, descr=GAMN, original_mnemonic=GAMN, data.shape=(2732,)),
 CurveItem(mnemonic=NEUT, unit=CPS, value=, descr=NEUT, original_mnemonic=NEUT, data.shape=(2732,)),
 CurveItem(mnemonic=PR, unit=OHM/M, value=, descr=PR, original_mnemonic=PR, data.shape=(2732,)),
 CurveItem(mnemonic=SP, unit=MV, value=, descr=SP, original_mnemonic=SP, data.shape=(2732,)),
 CurveItem(mnemonic=COND, unit=MS/M, value=, descr=COND, original_mnemonic=COND, data.shape=(2732,))]

## Header section metadata

#### lasio is able to reliably parse LAS header sections

The header sections are stored in the dictionary las.sections:

In [20]:
type(las.sections)

las.sections.keys()


dict_keys(['Version', 'Well', 'Curves', 'Parameter', 'Other'])

In [21]:
las.sections['Version']

[HeaderItem(mnemonic=VERS, unit=, value=2.0, descr=CWLS LOG ASCII STANDARD - ...),
 HeaderItem(mnemonic=WRAP, unit=, value=NO, descr=ONE LINE PER DEPTH STEP)]

In [22]:
las.version

[HeaderItem(mnemonic=VERS, unit=, value=2.0, descr=CWLS LOG ASCII STANDARD - ...),
 HeaderItem(mnemonic=WRAP, unit=, value=NO, descr=ONE LINE PER DEPTH STEP)]

 same for 
  * LASFile.version and LASFile.sections['Version']
  * LASFile.well and LASFile.sections['Well']
  * LASFile.curves and LASFile.sections['Curves']
  * LASFile.params and LASFile.sections['Parameter']
  * LASFile.other and LASFile.sections['Other']
  * LASFile.sections['extra section']
  * LASFile.data or each column is in LASFile.curves[...].data

In [23]:
las.data

array([[ 5.00000e-02,  4.97650e+01,  4.58700e+00, ...,          nan,
                 nan,          nan],
       [ 1.00000e-01,  4.97650e+01,  4.58700e+00, ...,  1.15508e+02,
        -3.04900e+00, -1.16998e+02],
       [ 1.50000e-01,  4.97650e+01,  4.58700e+00, ...,  1.15508e+02,
        -3.04900e+00, -1.16998e+02],
       ...,
       [ 1.36500e+02,  4.85550e+01,          nan, ...,          nan,
                 nan,          nan],
       [ 1.36550e+02,  4.84380e+01,          nan, ...,          nan,
                 nan,          nan],
       [ 1.36600e+02, -5.62750e+01,          nan, ...,          nan,
                 nan,          nan]])

In [24]:
las.curves

[CurveItem(mnemonic=DEPT, unit=M, value=, descr=DEPTH, original_mnemonic=DEPT, data.shape=(2732,)),
 CurveItem(mnemonic=CALI, unit=MM, value=, descr=CALI, original_mnemonic=CALI, data.shape=(2732,)),
 CurveItem(mnemonic=DFAR, unit=G/CM3, value=, descr=DFAR, original_mnemonic=DFAR, data.shape=(2732,)),
 CurveItem(mnemonic=DNEAR, unit=G/CM3, value=, descr=DNEAR, original_mnemonic=DNEAR, data.shape=(2732,)),
 CurveItem(mnemonic=GAMN, unit=GAPI, value=, descr=GAMN, original_mnemonic=GAMN, data.shape=(2732,)),
 CurveItem(mnemonic=NEUT, unit=CPS, value=, descr=NEUT, original_mnemonic=NEUT, data.shape=(2732,)),
 CurveItem(mnemonic=PR, unit=OHM/M, value=, descr=PR, original_mnemonic=PR, data.shape=(2732,)),
 CurveItem(mnemonic=SP, unit=MV, value=, descr=SP, original_mnemonic=SP, data.shape=(2732,)),
 CurveItem(mnemonic=COND, unit=MS/M, value=, descr=COND, original_mnemonic=COND, data.shape=(2732,))]

In [25]:
las.curves[CALI].data

NameError: name 'CALI' is not defined

Sections themselves are represented by lasio.las_items.SectionItems objects. This is a list which has been extended to allow you to access the items within by their mnemonic:

The 3 are equivalent
  * las.version.VERS
  * las.version['VERS']
  * las.version[0]

In [26]:
las.version.VERS

HeaderItem(mnemonic=VERS, unit=, value=2.0, descr=CWLS LOG ASCII STANDARD - ...)

#### las.well

In [27]:
las.well

[HeaderItem(mnemonic=STRT, unit=M, value=0.05, descr=FIRST INDEX VALUE),
 HeaderItem(mnemonic=STOP, unit=M, value=136.6, descr=LAST INDEX VALUE),
 HeaderItem(mnemonic=STEP, unit=M, value=0.05, descr=STEP),
 HeaderItem(mnemonic=NULL, unit=, value=-99999, descr=NULL VALUE),
 HeaderItem(mnemonic=COMP, unit=, value=, descr=COMP),
 HeaderItem(mnemonic=WELL, unit=, value=Scorpio E1, descr=WELL),
 HeaderItem(mnemonic=FLD, unit=, value=, descr=),
 HeaderItem(mnemonic=LOC, unit=, value=Mt Eba, descr=LOC),
 HeaderItem(mnemonic=SRVC, unit=, value=, descr=),
 HeaderItem(mnemonic=CTRY, unit=, value=, descr=),
 HeaderItem(mnemonic=STAT, unit=, value=SA, descr=STAT),
 HeaderItem(mnemonic=CNTY, unit=, value=, descr=),
 HeaderItem(mnemonic=DATE, unit=, value=15/03/2015, descr=DATE),
 HeaderItem(mnemonic=UWI, unit=, value=6038-187, descr=WUNT)]

In [28]:
las.well.CTRY = 'Australia'

In [29]:
las.well.CTRY

HeaderItem(mnemonic=CTRY, unit=, value=Australia, descr=)

#### set any of the attributes directly. 

In [30]:
las.curves.PR.unit = 'ohmm'

In [31]:
las.curves.PR

CurveItem(mnemonic=PR, unit=ohmm, value=, descr=PR, original_mnemonic=PR, data.shape=(2732,))

### Now let’s look more closely at how to manipulate and add or remove items from a section.



In [32]:
las.params

[HeaderItem(mnemonic=BS, unit=, value=216 mm, descr=BS),
 HeaderItem(mnemonic=JOBN, unit=, value=, descr=JOBN),
 HeaderItem(mnemonic=WPMT, unit=, value=, descr=WPMT),
 HeaderItem(mnemonic=AGL, unit=, value=, descr=AGL),
 HeaderItem(mnemonic=PURP, unit=, value=Cased hole stratigraphy, descr=PURP),
 HeaderItem(mnemonic=X, unit=, value=560160, descr=X),
 HeaderItem(mnemonic=CSGL, unit=, value=0 m - 135 m, descr=CSGL),
 HeaderItem(mnemonic=UNIT, unit=, value=, descr=UNIT),
 HeaderItem(mnemonic=Y, unit=, value=6686430, descr=Y),
 HeaderItem(mnemonic=TDL, unit=, value=135.2 m, descr=TDL),
 HeaderItem(mnemonic=PROD, unit=, value=, descr=PROD),
 HeaderItem(mnemonic=MUD, unit=, value=Water, descr=MUD),
 HeaderItem(mnemonic=CSGS, unit=, value=100 mm, descr=CSGS),
 HeaderItem(mnemonic=ENG, unit=, value=, descr=ENG),
 HeaderItem(mnemonic=STEP, unit=, value=5 cm, descr=STEP),
 HeaderItem(mnemonic=FLUIDLEVEL, unit=, value=54 m, descr=FluidLevel),
 HeaderItem(mnemonic=CSGT, unit=, value=PVC, descr=CS

*We want to rename the DREF mnemonic as LMF. We can do so by changing the header_item.mnemonic attribute*

In [33]:
las.params.DREF.mnemonic = 'LMF'

*And now we need to add a new mnemonic. Adding via an attribute will not work. You need to use the item-style access.*

In [ ]:
las.params['DRILL'] = lasio.HeaderItem(mnemonic='DRILL', value='John Smith', descr='Driller on site')